# Hyperspace Jump Drive

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import string
from sklearn.cluster import KMeans
from scipy.stats import pearsonr

import pyvisa as visa
import pandas as pd
from tqdm import tqdm
import time
import os

In [303]:
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'

In [ ]:
%run "../setup/Setup_Generic.ipynb"

INFO: Found ChipWhisperer😍


In [ ]:
cw.program_target(scope, prog, "hyperspaceJumpDrive-{}.hex".format(PLATFORM))

Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 13919 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 13919 bytes


In [ ]:
folder = "./Traces/hyperspaceJumpDrive/"  # Change this to your desired folder

if not os.path.exists(folder):
    print("folder done")
    os.makedirs(folder)


## Capture and Save Power Traces

In [ ]:
import random

flag_length = 12

plaintexts = []
power_traces = []

# Create a list of 0–255
values = list(range(256))

# Shuffle to get a random order
random.shuffle(values)

for i in range(256):

    # Define empty array to store the recovered flag characters
    plaintext = bytes([values[i]])
    plaintexts.append(values[i])
    print("Plaintext :",plaintext)

    # Arm the Chipwhisper
    scope.arm()

    target.simpleserial_write('p', plaintext)

    # Capture the power trace
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    # Read the device response
    response = target.simpleserial_read('r', 1)

    if response is not None:
        print(response)

    # Read the power trace from the ChipWhisperer Nano
    power_trace = scope.get_last_trace()

    # Append the power trace to the list of power traces
    power_traces.append(power_trace)

traces = np.array(power_traces)
data = {'power_traces': traces, "plaintexts":plaintexts}

np.savez("./Traces/hyperspaceJumpDrive/power_traces_and_plaintexts.npz", **data)

Plaintext : b'\x1f'
bytearray(b'\x01')
Plaintext : b'\xf5'
bytearray(b'\x01')
Plaintext : b'\xc5'
bytearray(b'\x01')
Plaintext : b'\xb1'
bytearray(b'\x01')
Plaintext : b'\xca'
bytearray(b'\x01')
Plaintext : b'\xc8'
bytearray(b'\x01')
Plaintext : b'w'
bytearray(b'\x01')
Plaintext : b'\x05'
bytearray(b'\x01')
Plaintext : b'\xb4'
bytearray(b'\x01')
Plaintext : b'\xa7'
bytearray(b'\x01')
Plaintext : b'R'
bytearray(b'\x01')
Plaintext : b'j'
bytearray(b'\x01')
Plaintext : b'\x1a'
bytearray(b'\x01')
Plaintext : b'#'
bytearray(b'\x01')
Plaintext : b'`'
bytearray(b'\x01')
Plaintext : b'\xda'
bytearray(b'\x01')
Plaintext : b'\xaa'
bytearray(b'\x01')
Plaintext : b'V'
bytearray(b'\x01')
Plaintext : b'c'
bytearray(b'\x01')
Plaintext : b'\xf7'
bytearray(b'\x01')
Plaintext : b'\xa5'
bytearray(b'\x01')
Plaintext : b' '
bytearray(b'\x01')
Plaintext : b'\x12'
bytearray(b'\x01')
Plaintext : b'C'
bytearray(b'\x01')
Plaintext : b'\xf3'
bytearray(b'\x01')
Plaintext : b'v'
bytearray(b'\x01')
Plaintext : b'\x

In [ ]:

try:
    input_file = './Traces/hyperspaceJumpDrive/power_traces_and_plaintexts.npz'
    data = np.load(input_file)
except FileNotFoundError:
    print(f"Error: File not found at '{input_file}'.")
    print("Make sure you run the capture and save script first.")
    exit()

power_traces = data['power_traces']
plaintexts = data['plaintexts']

print(f"✅ Data loaded successfully from '{input_file}'.")
print(f"Found {len(loaded_traces)} traces to visualize.")


# Calculate the Hamming weight and Hamming distance for the plaintexts for the subtraction operation (comparing two characters)
# Use a key character guess from the ASCII table and iterate through all characters to find the best correlation
# Guessing the key character (k)
# Plaintext character (p)
# HD = HW(p - k, p) 
def hamming_weight(x):
    return bin(x).count('1')

def hamming_distance(x, y):
    return hamming_weight(x ^ y)
key_guesses = range(256)  

pass_lenght = 12

recover_pass = []

for pos in range(0, pass_lenght):

    temp_power_traces = power_traces[:,0+(171*pos):50+(171*pos)]
    num_samples = len(temp_power_traces[0])
    correlation_results = np.zeros((len(key_guesses), num_samples))

    for k_index, k in enumerate(key_guesses):
        # Calculate Hamming distance for each plaintext with the key guess
        hamming_distances = np.array(
        [
            hamming_distance(k, pt) 
            for pt in plaintexts
        ]
        )

        power_model = hamming_distances

        for t in range(num_samples):
            correlation_results[k_index, t] = pearsonr(power_model, temp_power_traces[:, t])[0]

    # Find the key guess with the highest absolute correlation at any time sample
    correlation_results = np.nan_to_num(correlation_results)
    max_correlation = np.nanmax((correlation_results))
    print(f"Max Correlation Found: {max_correlation:.4f}")

    best_k_index, best_t = np.unravel_index(np.nanargmax((correlation_results)), correlation_results.shape)
    best_k = key_guesses[best_k_index]
    print(f"Best Key Guess: '{(best_k)}({hex(best_k)})' with Max Correlation: {correlation_results[best_k_index, best_t]:.4f} at Time Sample: {best_t}")

    # plt.plot(correlation_results)
    # plt.show()

    recover_pass.append(best_k)

print("Correlation analysis complete. ")


✅ Data loaded successfully from '/home/dakshina/projects/csaw_esc_2025/Traces/hyperspaceJumpDrive/power_traces_and_plaintexts.npz'.
Found 256 traces to visualize.
Max Correlation Found: 0.8722
Best Key Guess: '3(0x3)' with Max Correlation: 0.8722 at Time Sample: 24
Max Correlation Found: 0.8683
Best Key Guess: '125(0x7d)' with Max Correlation: 0.8683 at Time Sample: 24
Max Correlation Found: 0.8759
Best Key Guess: '57(0x39)' with Max Correlation: 0.8759 at Time Sample: 24
Max Correlation Found: 0.8686
Best Key Guess: '93(0x5d)' with Max Correlation: 0.8686 at Time Sample: 24
Max Correlation Found: 0.8935
Best Key Guess: '180(0xb4)' with Max Correlation: 0.8935 at Time Sample: 24
Max Correlation Found: 0.8729
Best Key Guess: '207(0xcf)' with Max Correlation: 0.8729 at Time Sample: 24
Max Correlation Found: 0.8904
Best Key Guess: '86(0x56)' with Max Correlation: 0.8904 at Time Sample: 24
Max Correlation Found: 0.8582
Best Key Guess: '102(0x66)' with Max Correlation: 0.8582 at Time Sample

In [313]:
guess_flag = bytes(recover_pass)
print(guess_flag)
target.simpleserial_write('a', guess_flag)
response = target.simpleserial_read('r', 17)
if response is not None:
    print(response)

b"\x03}9]\xb4\xcfVfo'\xbf["
bytearray(b'b64{c3RhcmR1c3Q=}')
